In [1]:
import pandas as pd
import csv
import json
monthID = "0623"
startMonthID = "0423"
endMonthID = "0623"

In [2]:
with open(f'C:/Users/weijin.ang/Code/Transactions/resi_transactions_{startMonthID}_{endMonthID}.json', 'r') as f:
  data = json.load(f)
#data

In [3]:
# normalize events
df = pd.json_normalize(data, 'Result')

# explode all columns with lists of dicts
df = df.apply(lambda x: x.explode()).reset_index(drop=True)  # df.apply(pd.Series.explode).reset_index(drop=True) also works

# list of columns with dicts
cols_to_normalize = ['rental']

# if there are keys, which will become column names, overlap with excising column names
# add the current column name as a prefix
normalized = list()
for col in cols_to_normalize:
    
    d = pd.json_normalize(df[col], sep='_')
    d.columns = [f'{col}_{v}' for v in d.columns]
    normalized.append(d.copy())

# combine df with the normalized columns , concat both lists
df = pd.concat([df] + normalized, axis=1).drop(columns=cols_to_normalize)

In [4]:
df
df.to_csv(f'C:/Users/weijin.ang/Code/Transactions/transaction_resi_converted_raw_csv_{startMonthID}_{endMonthID}.csv', index=False)
df

,street,x,project,y,rental_areaSqm,rental_leaseDate,rental_propertyType,rental_district,rental_areaSqft,rental_noOfBedRoom,rental_rent
0,ELLIOT ROAD,38865.17182,ELLIOT AT THE EAST COAST,32621.92134,210-220,0623,Non-landed Properties,15,2300-2400,3,6500
1,ELLIOT ROAD,38865.17182,ELLIOT AT THE EAST COAST,32621.92134,240-250,0523,Non-landed Properties,15,2600-2700,4,11000
2,ELLIOT ROAD,38865.17182,ELLIOT AT THE EAST COAST,32621.92134,120-130,0623,Non-landed Properties,15,1300-1400,3,5200
3,ELLIOT ROAD,38865.17182,ELLIOT AT THE EAST COAST,32621.92134,90-100,0523,Non-landed Properties,15,900-1000,2,4700
4,ELLIOT ROAD,38865.17182,ELLIOT AT THE EAST COAST,32621.92134,210-220,0623,Non-landed Properties,15,2300-2400,3,8050
...,...,...,...,...,...,...,...,...,...,...,...
20181,PERUMAL ROAD,30477.87296,UPTOWN @ FARRER,32796.58779,60-70,0423,Non-landed Properties,08,600-700,3,5600
20182,PERUMAL ROAD,30477.87296,UPTOWN @ FARRER,32796.58779,70-80,0423,Non-landed Properties,08,800-900,3,5500
20183,PERUMAL ROAD,30477.87296,UPTOWN @ FARRER,32796.58779,60-70,0623,Non-landed Properties,08,700-800,3,5100
20184,PERUMAL ROAD,30477.87296,UPTOWN @ FARRER,32796.58779,40-50,0623,Non-landed Properties,08,500-600,2,4000


In [5]:
df = df[df.rental_propertyType.isin(["Non-landed Properties", "Executive Condominium"])]
df = df[df.rental_noOfBedRoom.isin(["1","2","3","4","5"])]
df = df[df.rental_leaseDate.isin([monthID])]
df

,street,x,project,y,rental_areaSqm,rental_leaseDate,rental_propertyType,rental_district,rental_areaSqft,rental_noOfBedRoom,rental_rent
0,ELLIOT ROAD,38865.17182,ELLIOT AT THE EAST COAST,32621.92134,210-220,0623,Non-landed Properties,15,2300-2400,3,6500
2,ELLIOT ROAD,38865.17182,ELLIOT AT THE EAST COAST,32621.92134,120-130,0623,Non-landed Properties,15,1300-1400,3,5200
4,ELLIOT ROAD,38865.17182,ELLIOT AT THE EAST COAST,32621.92134,210-220,0623,Non-landed Properties,15,2300-2400,3,8050
9,HILLVIEW AVENUE,19806.57039,MERAWOODS,37591.36252,90-100,0623,Non-landed Properties,23,1000-1100,2,3300
10,HILLVIEW AVENUE,19806.57039,MERAWOODS,37591.36252,120-130,0623,Non-landed Properties,23,1300-1400,3,4200
...,...,...,...,...,...,...,...,...,...,...,...
20173,PUNGGOL FIELD,35969.63449,PRIVE,42560.406,70-80,0623,Executive Condominium,19,700-800,2,4300
20174,PUNGGOL FIELD,35969.63449,PRIVE,42560.406,70-80,0623,Executive Condominium,19,800-900,2,3850
20179,PERUMAL ROAD,30477.87296,UPTOWN @ FARRER,32796.58779,60-70,0623,Non-landed Properties,08,700-800,3,5000
20183,PERUMAL ROAD,30477.87296,UPTOWN @ FARRER,32796.58779,60-70,0623,Non-landed Properties,08,700-800,3,5100


In [6]:
#df_50th = df.groupby(['rental_district']).mean()
df_25th = df.groupby(['rental_district', 'rental_noOfBedRoom']).quantile(.25).rename(columns={"rental_rent": "25th_rental"})
df_50th = df.groupby(['rental_district', 'rental_noOfBedRoom']).quantile(.50).rename(columns={"rental_rent": "50th_rental"})
df_75th = df.groupby(['rental_district', 'rental_noOfBedRoom']).quantile(.75).rename(columns={"rental_rent": "75th_rental"})

C:\Users\weijin.ang\AppData\Local\Temp\ipykernel_17904\1960684038.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.quantile is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_25th = df.groupby(['rental_district', 'rental_noOfBedRoom']).quantile(.25).rename(columns={"rental_rent": "25th_rental"})
C:\Users\weijin.ang\AppData\Local\Temp\ipykernel_17904\1960684038.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.quantile is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_50th = df.groupby(['rental_district', 'rental_noOfBedRoom']).quantile(.50).rename(columns={"rental_rent": "50th_rental"})
C:\Users\weijin.ang\AppData\Local\Temp\ipykernel_17904\1960684038.py:4: FutureWarning: The default value of numeric_only

In [7]:
df = pd.concat([df_25th] + [df_50th] + [df_75th], axis=1)
df

25th_rental  50th_rental  75th_rental
rental_district rental_noOfBedRoom                                       
01              1                        4425.0       5000.0       5537.5
                2                        6000.0       7100.0       7700.0
                3                        7975.0       8950.0      10750.0
                4                       12000.0      13800.0      17500.0
02              1                        4000.0       4368.0       4900.0
...                                         ...          ...          ...
27              4                        4475.0       4550.0       4850.0
28              1                        2650.0       2900.0       3050.0
                2                        3300.0       3450.0       3562.5
                3                        4000.0       4500.0       4700.0
                4                        4575.0       4750.0       5000.0

[114 rows x 3 columns]

In [8]:
df = pd.pivot_table(df, values = ['25th_rental','50th_rental','75th_rental'], index=['rental_district'], columns = 'rental_noOfBedRoom').reset_index()
df


rental_district 25th_rental                            \
rental_noOfBedRoom                           1       2        3        4   
0                               01     4425.00  6000.0  7975.00  12000.0   
1                               02     4000.00  4841.5  4819.00  16000.0   
2                               03     3700.00  4700.0  6000.00   8875.0   
3                               04     3400.00  4625.0  7950.00  12000.0   
4                               05     3200.00  4000.0  5225.00   6250.0   
5                               06     5875.25     NaN      NaN      NaN   
6                               07     3900.00  4000.0  5600.00  12500.0   
7                               08     3000.00  4075.0  5000.00   5625.0   
8                               09     3600.00  5200.0  6000.00   9875.0   
9                               10     3425.00  4550.0  6300.00   9950.0   
10                              11     3125.00  4400.0  5500.00   8075.0   
11                              12     3000.00  3500.0  4000.00   6150.0   
12                              13     3100.00  4000.0  4500.00   5700.0   
13                              14     2800.00  3700.0  4200.00   5500.0   
14                              15     3000.00  3700.0  4800.00   6125.0   
15                              16     3000.00  3450.0  4494.00   4650.0   
16                              17     2900.00  3200.0  3900.00   4950.0   
17                              18     2850.00  3500.0  4000.00   5000.0   
18                              19     2800.00  3500.0  4143.75   5100.0   
19                              20     3050.00  3900.0  4800.00   6762.5   
20                              21     2700.00  3600.0  4300.00   5000.0   
21                              22     3175.00  3900.0  4500.00   5550.0   
22                              23     2875.00  3400.0  3987.50   4950.0   
23                              25     2587.50  3200.0  4000.00   5000.0   
24                              26     3000.00  3300.0  3850.00   4300.0   
25                              27     2800.00  3375.0  3900.00   4475.0   
26                              28     2650.00  3300.0  4000.00   4575.0   

                            50th_rental                                    \
rental_noOfBedRoom        5           1       2       3        4        5   
0                       NaN      5000.0  7100.0  8950.0  13800.0      NaN   
1                       NaN      4368.0  5300.0  7300.0  16000.0      NaN   
2                   11000.0      4000.0  5150.0  6750.0   9050.0  11000.0   
3                   26750.0      3600.0  5500.0  9450.0  14500.0  28500.0   
4                       NaN      3500.0  4300.0  5950.0   7000.0      NaN   
5                       NaN      6183.5     NaN     NaN      NaN      NaN   
6                       NaN      5000.0  4750.0  6000.0  12500.0      NaN   
7                       NaN      3380.0  4500.0  5500.0   5950.0      NaN   
8                   10050.0      4000.0  5800.0  7900.0  12500.0  11300.0   
9                   18125.0      3675.0  5300.0  7400.0  13000.0  24000.0   
10                      NaN      3400.0  5000.0  6550.0   9430.0      NaN   
11                      NaN      3200.0  3800.0  4500.0   6500.0      NaN   
12                      NaN      3300.0  4225.0  5200.0   7100.0      NaN   
13                      NaN      3000.0  4025.0  4600.0   6000.0      NaN   
14                   6800.0      3300.0  4400.0  6000.0   8300.0   6800.0   
15                   8700.0      3200.0  3765.0  5000.0   6150.0   8700.0   
16                      NaN      3000.0  3500.0  4025.0   5400.0      NaN   
17                   6625.0      3000.0  3700.0  4375.0   5400.0   7250.0   
18                   6575.0      3000.0  3800.0  4500.0   5800.0   7450.0   
19                  10000.0      3450.0  4200.0  5250.0   7250.0  10000.0   
20                      NaN      2900.0  4000.0  5000.0   5950.0      NaN   
21                      NaN      3300.0  42

In [9]:
df.columns = df.columns.swaplevel(0, 1)
df.sort_index(axis=1, level=0, inplace=True)
df

rental_noOfBedRoom                           1                          \
                   rental_district 25th_rental 50th_rental 75th_rental   
0                               01     4425.00      5000.0     5537.50   
1                               02     4000.00      4368.0     4900.00   
2                               03     3700.00      4000.0     4300.00   
3                               04     3400.00      3600.0     4500.00   
4                               05     3200.00      3500.0     3637.50   
5                               06     5875.25      6183.5     6491.75   
6                               07     3900.00      5000.0     6000.00   
7                               08     3000.00      3380.0     3800.00   
8                               09     3600.00      4000.0     4775.00   
9                               10     3425.00      3675.0     4000.00   
10                              11     3125.00      3400.0     3700.00   
11                              12     3000.00      3200.0     3450.00   
12                              13     3100.00      3300.0     3500.00   
13                              14     2800.00      3000.0     3300.00   
14                              15     3000.00      3300.0     3700.00   
15                              16     3000.00      3200.0     3300.00   
16                              17     2900.00      3000.0     3000.00   
17                              18     2850.00      3000.0     3100.00   
18                              19     2800.00      3000.0     3100.00   
19                              20     3050.00      3450.0     3775.00   
20                              21     2700.00      2900.0     3300.00   
21                              22     3175.00      3300.0     3512.50   
22                              23     2875.00      3075.0     3400.00   
23                              25     2587.50      2650.0     2812.50   
24                              26     3000.00      3000.0     3000.00   
25                              27     2800.00      3000.0     3350.00   
26                              28     2650.00      2900.0     3050.00   

rental_noOfBedRoom           2                                   3  \
                   25th_rental 50th_rental 75th_rental 25th_rental   
0                       6000.0      7100.0      7700.0     7975.00   
1                       4841.5      5300.0      5850.0     4819.00   
2                       4700.0      5150.0      5600.0     6000.00   
3                       4625.0      5500.0      6000.0     7950.00   
4                       4000.0      4300.0      4600.0     5225.00   
5                          NaN         NaN         NaN         NaN   
6                       4000.0      4750.0      6194.5     5600.00   
7                       4075.0      4500.0      4762.5     5000.00   
8                       5200.0      5800.0      6800.0     6000.00   
9                       4550.0      5300.0      6000.0     6300.00   
10                      4400.0      5000.0      5850.0     5500.00   
11                      3500.0      3800.0      4287.5     4000.00   
12                      4000.0      4225.0      4500.0     4500.00   
13                      3700.0      4025.0      4625.0     4200.00   
14                      3700.0      4400.0      5000.0     4800.00   
15                      3450.0      3765.0      4000.0     4494.00   
16                      3200.0      3500.0      3600.0     3900.00   
17                      3500.0      3700.0      4000.0     4000.00   
18                      3500.0      3800.0      4025.0     4143.75   
19                      3900.0      4200.0      4500.0     4800.00   
20                      3600.0      4000.0      4350.0     4300.00   
21                      3900.0      4200.0      4500.0     4500.00   
22                      3400.0      3700.0      4075.0     3987.50   
23                      3200.0      3400.0      3600.0     4000.00   
24                      3300.0   

In [10]:
df.to_csv(f'C:/Users/weijin.ang/Code/Transactions/transactions_resi_price_index_{monthID}.csv', na_rep='N/A', quoting=csv.QUOTE_NONE, index=False)